In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [2]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [3]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [4]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [5]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [154]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.27557887
====> Test set loss: 1.2546, 66.0%
====> Epoch: 150 Average loss: 1.24960347
====> Test set loss: 1.1862, 68.0%
====> Epoch: 225 Average loss: 1.21563738
====> Test set loss: 1.1894, 66.5%
====> Epoch: 300 Average loss: 1.19709698
====> Test set loss: 1.1760, 67.5%
====> Epoch: 375 Average loss: 1.21821563
====> Test set loss: 1.1765, 68.0%
====> Epoch: 450 Average loss: 1.19930835
====> Test set loss: 1.1765, 68.0%
====> Epoch: 525 Average loss: 1.16457643
====> Test set loss: 1.1754, 68.0%
====> Epoch: 600 Average loss: 1.17904321
====> Test set loss: 1.1766, 68.0%
====> Epoch: 675 Average loss: 1.17668282
====> Test set loss: 1.1764, 68.0%
====> Epoch: 750 Average loss: 1.24575864
====> Test set loss: 1.1767, 68.0%
Training state:  False
Elapsed:  50.90697503089905
Complete set accuracy: 72.89999999999999%


### Runner

In [8]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(100, 150):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 100
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.24946000
====> Test set loss: 1.2595, 70.0%
====> Epoch: 150 Average loss: 1.19981882
====> Test set loss: 1.2411, 69.5%
====> Epoch: 225 Average loss: 1.19483239
====> Test set loss: 1.2448, 69.5%
====> Epoch: 300 Average loss: 1.17696313
====> Test set loss: 1.2464, 70.0%
====> Epoch: 375 Average loss: 1.20221474
====> Test set loss: 1.2472, 69.5%
====> Epoch: 450 Average loss: 1.18685121
====> Test set loss: 1.2464, 69.5%
====> Epoch: 525 Average loss: 1.16002983
====> Test set loss: 1.2454, 69.5%
====> Epoch: 600 Average loss: 1.17463737
====> Test set loss: 1.2458, 69.5%
====> Epoch: 675 Average loss: 1.17455292
====> Test set loss: 1.2460, 69.5%
====> Epoch: 750 Average loss: 1.27622917
====> Test set loss: 1.2445, 69.5%
Training state:  False
Complete set accuracy: 73.8%
Log accuracy: 72.39999999999999%
---- Done in  47.944681882858276  seconds

---- Running for model name:  B_ad

====> Epoch: 675 Average loss: 1.12561563
====> Test set loss: 1.1395, 74.5%
====> Epoch: 750 Average loss: 1.14485766
====> Test set loss: 1.1394, 75.0%
Training state:  False
Complete set accuracy: 74.9%
Log accuracy: 72.5%
---- Done in  58.05666375160217  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.27476753
====> Test set loss: 1.2270, 65.5%
====> Epoch: 150 Average loss: 1.20214863
====> Test set loss: 1.1326, 72.0%
====> Epoch: 225 Average loss: 1.20731553
====> Test set loss: 1.1463, 71.0%
====> Epoch: 300 Average loss: 1.19609477
====> Test set loss: 1.1357, 71.0%
====> Epoch: 375 Average loss: 1.17795593
====> Test set loss: 1.1431, 71.0%
====> Epoch: 450 Average loss: 1.18491647
====> Test set loss: 1.1416, 71.0%
====> Epoch: 525 Average loss: 1.22021655
====> Test set loss: 1.1420, 71.0%
====> Epoch: 600 Average loss: 1.20779576
====> Test set loss: 1.1401, 71.0%
====> Epoch: 675 Average loss: 1.23533430
====> Test set loss: 1.1398, 7

====> Epoch: 450 Average loss: 1.14388111
====> Test set loss: 1.2149, 71.0%
====> Epoch: 525 Average loss: 1.17335943
====> Test set loss: 1.2152, 71.0%
====> Epoch: 600 Average loss: 1.17993166
====> Test set loss: 1.2159, 71.0%
====> Epoch: 675 Average loss: 1.15460339
====> Test set loss: 1.2158, 70.5%
====> Epoch: 750 Average loss: 1.10556674
====> Test set loss: 1.2159, 70.5%
Training state:  False
Complete set accuracy: 75.2%
Log accuracy: 73.2%
---- Done in  55.40653109550476  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.28171901
====> Test set loss: 1.1233, 74.5%
====> Epoch: 150 Average loss: 1.17097865
====> Test set loss: 1.0771, 75.0%
====> Epoch: 225 Average loss: 1.18396929
====> Test set loss: 1.0764, 75.5%
====> Epoch: 300 Average loss: 1.14896185
====> Test set loss: 1.0760, 74.5%
====> Epoch: 375 Average loss: 1.14557036
====> Test set loss: 1.0784, 75.0%
====> Epoch: 450 Average loss: 1.16008516
====> Test set loss: 1.

====> Epoch: 225 Average loss: 1.20609127
====> Test set loss: 1.0096, 76.5%
====> Epoch: 300 Average loss: 1.17101070
====> Test set loss: 1.0030, 78.0%
====> Epoch: 375 Average loss: 1.20607565
====> Test set loss: 0.9960, 78.5%
====> Epoch: 450 Average loss: 1.19633790
====> Test set loss: 0.9959, 78.5%
====> Epoch: 525 Average loss: 1.17766462
====> Test set loss: 0.9951, 78.5%
====> Epoch: 600 Average loss: 1.17845791
====> Test set loss: 0.9954, 78.5%
====> Epoch: 675 Average loss: 1.11944149
====> Test set loss: 0.9954, 78.5%
====> Epoch: 750 Average loss: 1.16013581
====> Test set loss: 0.9949, 78.5%
Training state:  False
Complete set accuracy: 74.8%
Log accuracy: 73.2%
---- Done in  53.238011837005615  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.29939364
====> Test set loss: 1.2917, 60.5%
====> Epoch: 150 Average loss: 1.26458689
====> Test set loss: 1.2408, 60.5%
====> Epoch: 225 Average loss: 1.25284558
====> Test set loss: 1.2

====> Epoch: 75 Average loss: 1.28002908
====> Test set loss: 1.2212, 66.0%
====> Epoch: 150 Average loss: 1.17818840
====> Test set loss: 1.1411, 68.0%
====> Epoch: 225 Average loss: 1.17765620
====> Test set loss: 1.1513, 66.5%
====> Epoch: 300 Average loss: 1.21728784
====> Test set loss: 1.1478, 67.0%
====> Epoch: 375 Average loss: 1.15430090
====> Test set loss: 1.1486, 66.5%
====> Epoch: 450 Average loss: 1.14057157
====> Test set loss: 1.1484, 66.5%
====> Epoch: 525 Average loss: 1.14174267
====> Test set loss: 1.1490, 66.5%
====> Epoch: 600 Average loss: 1.18009407
====> Test set loss: 1.1486, 66.5%
====> Epoch: 675 Average loss: 1.13206788
====> Test set loss: 1.1489, 66.5%
====> Epoch: 750 Average loss: 1.19091386
====> Test set loss: 1.1493, 66.5%
Training state:  False
Complete set accuracy: 72.1%
Log accuracy: 70.5%
---- Done in  51.846883058547974  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.28438152
====> Test set loss: 1.2588, 

...

====> Epoch: 525 Average loss: 1.21801431
====> Test set loss: 1.1441, 70.0%
====> Epoch: 600 Average loss: 1.21043198
====> Test set loss: 1.1440, 70.0%
====> Epoch: 675 Average loss: 1.14892552
====> Test set loss: 1.1435, 69.5%
====> Epoch: 750 Average loss: 1.13203314
====> Test set loss: 1.1437, 69.5%
Training state:  False
Complete set accuracy: 74.9%
Log accuracy: 69.19999999999999%
---- Done in  65.95302319526672  seconds



Starting run for Dataset 144
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.25942729
====> Test set loss: 1.2537, 65.5%
====> Epoch: 150 Average loss: 1.26057294
====> Test set loss: 1.2365, 66.5%
====> Epoch: 225 Average loss: 1.22028782
====> Test set loss: 1.2098, 68.0%
====> Epoch: 300 Average loss: 1.19207037
====> Test set loss: 1.2125, 67.5%
====> Epoch: 375 Average loss: 1.23698416
====> Test set loss: 1.2068, 68.5%
====> Epoch: 450 Average loss: 1.22644613
====> Test set loss: 1.2057, 68.5%
====> Epoch: 525 Average loss: 1.

====> Epoch: 300 Average loss: 1.23684197
====> Test set loss: 1.1926, 68.0%
====> Epoch: 375 Average loss: 1.23516157
====> Test set loss: 1.1931, 68.0%
====> Epoch: 450 Average loss: 1.23003407
====> Test set loss: 1.1920, 68.0%
====> Epoch: 525 Average loss: 1.25215285
====> Test set loss: 1.1919, 68.0%
====> Epoch: 600 Average loss: 1.19190321
====> Test set loss: 1.1927, 68.0%
====> Epoch: 675 Average loss: 1.23227331
====> Test set loss: 1.1940, 68.0%
====> Epoch: 750 Average loss: 1.21807685
====> Test set loss: 1.1931, 68.0%
Training state:  False
Complete set accuracy: 71.89999999999999%
Log accuracy: 71.0%
---- Done in  62.11496186256409  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.25951837
====> Test set loss: 1.1930, 70.5%
====> Epoch: 150 Average loss: 1.19050285
====> Test set loss: 1.1432, 73.0%
====> Epoch: 225 Average loss: 1.21991808
====> Test set loss: 1.1443, 73.0%
====> Epoch: 300 Average loss: 1.19661122
====> Test set lo

====> Epoch: 75 Average loss: 1.20117394
====> Test set loss: 1.1947, 66.5%
====> Epoch: 150 Average loss: 1.20873358
====> Test set loss: 1.1774, 64.0%
====> Epoch: 225 Average loss: 1.16984750
====> Test set loss: 1.1716, 64.0%
====> Epoch: 300 Average loss: 1.13474010
====> Test set loss: 1.1695, 64.0%
====> Epoch: 375 Average loss: 1.14909053
====> Test set loss: 1.1690, 63.5%
====> Epoch: 450 Average loss: 1.15217863
====> Test set loss: 1.1691, 63.5%
====> Epoch: 525 Average loss: 1.15140228
====> Test set loss: 1.1692, 63.5%
====> Epoch: 600 Average loss: 1.16125360
====> Test set loss: 1.1696, 63.5%
====> Epoch: 675 Average loss: 1.13921148
====> Test set loss: 1.1693, 63.5%
====> Epoch: 750 Average loss: 1.16000192
====> Test set loss: 1.1692, 63.5%
Training state:  False
Complete set accuracy: 70.89999999999999%
Log accuracy: 71.3%
---- Done in  66.35183811187744  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.25424586
====> Test 

====> Epoch: 750 Average loss: 1.09863596
====> Test set loss: 1.1100, 73.0%
Training state:  False
Complete set accuracy: 76.3%
Log accuracy: 74.6%
---- Done in  67.20487904548645  seconds

---- Running for model name:  F_mod_nadd_lin
====> Epoch: 75 Average loss: 1.23267644
====> Test set loss: 1.1694, 71.5%
====> Epoch: 150 Average loss: 1.12464893
====> Test set loss: 1.1113, 70.5%
====> Epoch: 225 Average loss: 1.14057149
====> Test set loss: 1.1100, 71.0%
====> Epoch: 300 Average loss: 1.15367754
====> Test set loss: 1.1144, 71.0%
====> Epoch: 375 Average loss: 1.14426760
====> Test set loss: 1.1163, 70.5%
====> Epoch: 450 Average loss: 1.10911561
====> Test set loss: 1.1165, 70.5%
====> Epoch: 525 Average loss: 1.15352104
====> Test set loss: 1.1159, 70.5%
====> Epoch: 600 Average loss: 1.13386280
====> Test set loss: 1.1170, 70.5%
====> Epoch: 675 Average loss: 1.16077183
====> Test set loss: 1.1166, 70.5%
====> Epoch: 750 Average loss: 1.18349198
====> Test set loss: 1.1162, 7

====> Epoch: 525 Average loss: 1.21092348
====> Test set loss: 1.1271, 74.5%
====> Epoch: 600 Average loss: 1.18103927
====> Test set loss: 1.1254, 74.5%
====> Epoch: 675 Average loss: 1.18000229
====> Test set loss: 1.1248, 74.0%
====> Epoch: 750 Average loss: 1.20308394
====> Test set loss: 1.1233, 74.5%
Training state:  False
Complete set accuracy: 73.3%
Log accuracy: 69.1%
---- Done in  65.86343502998352  seconds



Starting run for Dataset 149
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.15943873
====> Test set loss: 1.1467, 69.5%
====> Epoch: 150 Average loss: 1.11838897
====> Test set loss: 1.1285, 72.5%
====> Epoch: 225 Average loss: 1.13042230
====> Test set loss: 1.1297, 72.0%
====> Epoch: 300 Average loss: 1.13634050
====> Test set loss: 1.1314, 72.0%
====> Epoch: 375 Average loss: 1.14651569
====> Test set loss: 1.1300, 71.5%
====> Epoch: 450 Average loss: 1.08832149
====> Test set loss: 1.1300, 71.5%
====> Epoch: 525 Average loss: 1.10660243
====